In [3]:
from __future__ import absolute_import, division, print_function, unicode_literals
import functools
import seaborn as sns
import numpy as np
import tensorflow as tf
import pandas as pd
import os
import pymongo
import requests
import json
from datetime import date
from datetime import timedelta 

In [4]:
def get_team_features(team_name):
    client = pymongo.MongoClient("mongodb://192.168.99.100:27017")
    db = client['plp']
    dic = db["team"].find_one({"_id":{'$regex':'^'+team_name}})
    if(dic == None):
        return None
    del dic['_id']
    return np.fromiter(dic.values(),dtype = float)

def get_no_res_matches():
    client = pymongo.MongoClient("mongodb://192.168.99.100:27017")
    db = client['refined-database']
    return pd.DataFrame(db["new_matches"].find({}))

In [5]:
def print_logits (y,batch_size,nb_label):
    # Print a the label predicted
    #Input : 
    #y : Numpy array [batch_size][10] The data predicted
    #batch_size : Int The size of the batch
    #nb_label : Int The number of labels
    for i in range (0,batch_size):
        print("------------",i,"------------")
        for j in range (0,nb_label):
            if (y[i][j] != 0):
                print("prédiction : ",j,"pourcentage de certitude :","%.1f" % (y[i][j]*100) , " %")
                
def create_model():
    model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dense(3, activation='softmax')
  ])

    model.compile(optimizer='adam',
                loss='categorical_crossentropy',
                metrics=[
                      tf.keras.metrics.BinaryAccuracy(name='accuracy'),
                      tf.keras.metrics.Precision(name='precision'),
                      tf.keras.metrics.Recall(name='recall'),
                      tf.keras.metrics.AUC(name='auc')])

    return model


In [4]:
teams_to_predict = get_no_res_matches()

In [10]:
teams_to_predict.drop(columns=teams_to_predict.columns[3:],inplace = True)
teams_to_predict.drop(columns= teams_to_predict.columns[0],inplace = True)

In [17]:
def names_to_features(names):
    away = get_team_features(names[0])
    home = get_team_features(names[1])
    feature_vec = np.array([away, home])
    vec_out = feature_vec.reshape((1,np.prod(feature_vec.shape)))
    return vec_out

In [20]:
checkpoint_path = "models/model1.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

In [21]:
model = create_model()
model.load_weights(checkpoint_path)

In [26]:
for i in range(teams_to_predict.shape[0]):
    x_pred = names_to_features(teams_to_predict.iloc[i].values)
    print(teams_to_predict.iloc[i])
    if x_pred[0] is not None:
        print(x_pred)
        print_logits(model.predict(x = x_pred, batch_size = 1),1,3)

home    Tottenham
away     Brighton
Name: 0, dtype: object
[[-0.25122813  0.54554743  0.55778607 -0.92930384  0.28884216 -0.7193251
  -0.43959023  1.02777506  0.35266248  0.55778607 -0.78305012 -0.32699584
  -0.7193251  -0.43959023]]
------------ 0 ------------
prédiction :  0 pourcentage de certitude : 30.6  %
prédiction :  1 pourcentage de certitude : 32.1  %
prédiction :  2 pourcentage de certitude : 37.3  %
home      Aston
away    Norwich
Name: 1, dtype: object
[[ 1.02777506 -0.03310741 -0.24967723 -0.19803521  0.28884216  1.23042712
  -0.43959023  0.60144066  1.12420226  0.55778607  0.3869797   0.90468016
   1.23042712 -0.43959023]]
------------ 0 ------------
prédiction :  0 pourcentage de certitude : 24.4  %
prédiction :  1 pourcentage de certitude : 23.8  %
prédiction :  2 pourcentage de certitude : 51.8  %
home        Chelsea
away    Southampton
Name: 2, dtype: object
[[ 0.60144066  0.35266248  0.96151772 -0.49054266 -0.32699584 -0.23188705
  -0.43959023  1.02777506  0.5455474

ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type NoneType).

In [18]:
x_pred = names_to_features(teams_to_predict)


In [19]:
x_pred.shape

(1, 14)

Envoie des réponses à la base